# Step 4: Compute Derived Products and Export Model Inputs

With data ingested into the container, we now compute derived products:

1. **Fused NDVI** - Combine Landsat + Sentinel for better temporal coverage (if both available)
2. **Dynamics** - Irrigation windows, groundwater subsidy, K parameters

Then we export the `prepped_input.json` file that the SWIM model needs.

The container's `compute` and `export` APIs make this straightforward:
- `container.compute.fused_ndvi()` - Combine sensors
- `container.compute.dynamics()` - Compute irrigation/groundwater parameters
- `container.export.prepped_input_json()` - Export model-ready JSON

In [ ]:
import os
import sys
from pathlib import Path

root = os.path.abspath('../..')
sys.path.append(root)

from swimrs.container import open_container

## 1. Open the Container

In [ ]:
project_dir = Path.cwd()
container_path = project_dir / 'data' / '1_Boulder.swim'

container = open_container(str(container_path), mode='r+')

print(f"Opened container: {container.project_name}")
print(f"Fields: {container.n_fields}")

## 2. Check Current Status

Let's verify the data was ingested correctly in the previous notebook.

In [ ]:
print(container.query.status())

## 3. Compute Fused NDVI

If we have both Landsat and Sentinel NDVI, we can fuse them to get better temporal coverage. For this tutorial with only Landsat, the fused NDVI will simply be the Landsat NDVI.

In [ ]:
print("Computing fused NDVI...")
try:
    container.compute.fused_ndvi(
        masks=('irr', 'inv_irr'),
        instrument1='landsat',
        instrument2='sentinel',  # Will use Landsat only if Sentinel not available
        min_pairs=20,
        window_days=5,
        overwrite=True
    )
    print("Fused NDVI computed successfully")
except Exception as e:
    print(f"Note: Fused NDVI computation: {e}")
    print("Continuing with single-sensor NDVI...")

## 4. Compute Dynamics

This is the heart of SWIM-RS preprocessing. The dynamics computation:

- **Irrigation Windows**: Identifies periods when irrigation likely occurred based on ETf patterns
- **Groundwater Subsidy**: Detects fields receiving groundwater contributions to ET
- **K Parameters**: Computes ke_max (evaporation coefficient) and kc_max (crop coefficient)

The `use_mask=True` setting uses the annual irrigation fraction from IrrMapper/LANID to classify fields as irrigated vs non-irrigated.

In [ ]:
print("Computing dynamics...")
container.compute.dynamics(
    etf_model='ssebop',
    masks=('irr', 'inv_irr'),
    irr_threshold=0.3,  # Fields with >30% irrigated area are considered irrigated
    use_mask=True,      # Use irrigation mask from properties
    use_lulc=False,     # Don't compute irrigation from water balance (CONUS mode)
    lookback=5,         # Days to look back for irrigation window extension
    overwrite=True
)
print("Dynamics computed successfully")

## 5. Explore Computed Dynamics

Let's look at the irrigation windows for a sample field.

In [ ]:
# Get irrigation windows for a few fields
sample_fields = container.field_uids[:3]
try:
    irr_windows = container.compute.irrigation_windows(fields=sample_fields)
    
    for fid in sample_fields:
        if fid in irr_windows:
            print(f"\nField {fid}:")
            for year in ['2018', '2019', '2020']:
                if year in irr_windows[fid]:
                    doys = irr_windows[fid][year].get('irr_doys', [])
                    if doys:
                        print(f"  {year}: {len(doys)} irrigation days (DOY {min(doys)}-{max(doys)})")
                    else:
                        print(f"  {year}: No irrigation detected")
except Exception as e:
    print(f"Could not retrieve irrigation windows: {e}")

## 6. Visualize NDVI with Irrigation Periods

Let's plot NDVI time series with irrigation periods marked.

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

%matplotlib inline

In [ ]:
try:
    ndvi = container.to_xarray('remote_sensing/ndvi/landsat/irr')
    sample_site = container.field_uids[0]
    
    # Get 2020 data
    ndvi_2020 = ndvi.sel(site=sample_site, time='2020')
    
    fig, ax = plt.subplots(figsize=(14, 5))
    
    # Plot NDVI
    ax.plot(ndvi_2020.time, ndvi_2020, 'go', markersize=4, label='NDVI')
    
    # Mark irrigation season (approximate: DOY 150-270)
    ax.axvspan('2020-05-30', '2020-09-27', alpha=0.2, color='blue', label='Typical irrigation season')
    
    ax.set_ylabel('NDVI')
    ax.set_xlabel('Date')
    ax.set_title(f'NDVI Time Series with Irrigation Season - Field {sample_site} (2020)')
    ax.legend()
    ax.set_ylim(0, 1)
    plt.tight_layout()
    plt.show()
except Exception as e:
    print(f"Could not create visualization: {e}")

## 7. Check Container Status After Compute

In [ ]:
print(container.query.status())

## 8. Export Model Input JSON

Finally, export the `prepped_input.json` file that the SWIM model needs. This contains:
- Field properties (soils, area, irrigation fraction)
- Irrigation data (per-year irrigation windows)
- Time series data (meteorology, remote sensing)
- Field ordering

In [ ]:
output_path = project_dir / 'data' / 'prepped_input.json'

print("Exporting model input JSON...")
container.export.prepped_input_json(
    output_path=str(output_path),
    etf_model='ssebop',
    masks=('irr', 'inv_irr'),
    instrument='landsat',
    use_fused_ndvi=True,
    irr_threshold=0.3
)

print(f"\nExported to: {output_path}")
print(f"File size: {output_path.stat().st_size / 1024 / 1024:.1f} MB")

## 9. Verify Export

Let's peek at the exported JSON structure.

In [ ]:
import json

with open(output_path, 'r') as f:
    data = json.load(f)

print("Top-level keys:", list(data.keys()))
print(f"\nNumber of fields in 'order': {len(data.get('order', []))}")
print(f"Number of fields in 'props': {len(data.get('props', {}))}")
print(f"Number of dates in 'time_series': {len(data.get('time_series', {}))}")

# Show sample field properties
if data.get('props'):
    sample_fid = list(data['props'].keys())[0]
    print(f"\nSample field properties ({sample_fid}):")
    for k, v in data['props'][sample_fid].items():
        print(f"  {k}: {v}")

## 10. Save and Close

In [ ]:
container.save()
container.close()

print(f"Container saved.")
print(f"\nReady to run the model!")
print("Next: Run notebook 05 to execute the SWIM model and visualize outputs")